## get the china provinces shapely form

In [ ]:
# running the jupyter notebook on the compute nodes doesnt build the path as expected, you have to manually do this
import os

import IPython

working_directory = os.path.dirname(
    IPython.extract_module_locals()[1]["__vsc_ipynb_file__"]
)
os.chdir(os.path.dirname(os.path.join(working_directory, "scripts")))

In [ ]:
import os.path
import sys

import cartopy.io.shapereader as shpreader
import geopandas as gpd
import matplotlib.pyplot as plt

scripts_path = "../scripts/"
sys.path.append(os.path.abspath(scripts_path))

NATURAL_EARTH_RESOLUTION = "10m"  # 1:10m scale
NATURAL_EARTH_DATA_SET = "admin_1_states_provinces"
CRS = 4326  # WGS84

In [ ]:
gdf = gpd.read_file(
    "/home/ivanra/documents/PyPSA-China-PIK/resources/data/regions/regions_offshore_province.geojson"
)
with open(
    "/home/ivanra/documents/PyPSA-China-PIK/resources/data/regions/test.geojson", "w"
) as f:
    f.write(gdf.to_json(drop_id=True))
gdf.crs

In [ ]:
gdf2 = gpd.read_file(
    "/home/ivanra/documents/PyPSA-China-PIK/resources/data/regions/test.geojson"
)
gdf2.crs

In [ ]:
NATURAL_EARTH_DATA_SET = "ad"

In [ ]:
shpfilename = shpreader.natural_earth(
    resolution=NATURAL_EARTH_RESOLUTION, category="cultural", name="admin_0_countries"
)
reader = shpreader.Reader(shpfilename)
print("downloaded successfully")
countries = list(reader.records())

In [ ]:
[c for c in countries if c.attributes["SOVEREIGNT"] == "China"]

In [ ]:
def fetch_natural_earth_shape(
    dataset_name: str, filter_key: str, filter_value="China", region_key=None
) -> gpd.GeoDataFrame:
    """Fetch region or country shape from natural earth dataset and filter

    Args:
        dataset_name (str): the name of the natural earth dataset to fetch
        filter_key (str): key to filter the records by
        filter_value (str|list, optional): filter pass value. Defaults to "China".

    Example:
        china country: build_natural_earth_shape("admin_0_countries", "ADMIN", "China")
        china provinces: build_natural_earth_shape("admin_1_states_provinces", "iso_a2", "CN", region_key = "name_en")

    Returns:
        gpd.GeoDataFrame: the filtered records
    """
    shpfilename = shpreader.natural_earth(
        resolution=NATURAL_EARTH_RESOLUTION, category="cultural", name=dataset_name
    )
    reader = shpreader.Reader(shpfilename)
    records = list(reader.records())
    if not region_key:
        region_key = filter_key
    if isinstance(filter_value, list):
        gdf = gpd.GeoDataFrame(
            [
                {"region": c.attributes[region_key], "geometry": c.geometry}
                for c in records
                if c.attributes[filter_key] in filter_value
            ]
        )
    else:
        gdf = gpd.GeoDataFrame(
            [
                {"region": c.attributes[region_key], "geometry": c.geometry}
                for c in records
                if c.attributes[filter_key] == filter_value
            ]
        )
    gdf.set_crs(epsg=CRS, inplace=True)
    return gdf

In [ ]:
fig, ax = plt.subplots()
china_shape = fetch_natural_earth_shape("admin_0_countries", "ADMIN", "China")
china_shape.plot(ax=ax)
gpd.read_file(
    "/home/ivanra/documents/PyPSA-China-PIK/resources/data/regions/regions_offshore_province.geojson"
).plot(ax=ax, color="red", alpha=0.5)
gpd.read_file(
    "/home/ivanra/documents/PyPSA-China-PIK/resources/data/regions/regions_offshore.geojson"
).plot(ax=ax, color="green", alpha=0.5)

In [ ]:
marim = gpd.read_file(
    "/home/ivanra/documents/PyPSA-China-PIK/resources/data/regions/regions_offshore_province.geojson"
)
marim.plot(lw=3)

In [ ]:
province_shapes = fetch_natural_earth_shape(
    "admin_1_states_provinces", "iso_a2", "CN", region_key="name_en"
)
fig, ax = plt.subplots(figsize=(10, 10))
province_shapes.plot(ax=ax)

center_point = lambda poly: (poly.centroid.xy[0][0], poly.centroid.xy[1][0])
for idx, row in province_shapes.iterrows():
    plt.annotate(
        text=row["region"],
        xy=center_point(row["geometry"]),
        horizontalalignment="center",
    )

In [ ]:
import requests


def fetch_maritime_eez(zone_name: str) -> gpd.GeoDataFrame:
    """Fetch maritime data for a country from Maritime Gazette API#
    (Royal marine institute of Flanders data base)

    Args:
        zone_name (str): the country's zone name, e.g "Chinese" for china

    Raises:
        requests.HTTPError: if the request fails
    Returns:
        dict: the maritime data
    """

    def find_record_id(zone_name: str) -> int:
        # get Maritime Gazette record ID for the country
        # the eez ID is 70: see https://www.marineregions.org/gazetteer.php?p=webservices&type=rest#/
        url = f"https://www.marineregions.org/rest/getGazetteerRecordsByName.json/{zone_name}/?like=true&fuzzy=false&typeID=70&offset=0&count=100"
        response = requests.get(url)
        if response.status_code != 200:
            raise requests.HTTPError(
                f"Failed to retrieve Maritime Gazette ID. Status code: {response.status_code}"
            )
        record_data = response.json()
        print(record_data)
        return [
            data
            for data in record_data
            if (data["status"] == "standard")
            and (data["preferredGazetteerName"].lower().find(zone_name.lower()) != -1)
        ][0]["MRGID"]

    mgrid = find_record_id(zone_name)

    #  URL of the WFS service
    url = "https://geo.vliz.be/geoserver/wfs"
    # WFS request parameters + record ID filter
    params = dict(
        service="WFS",
        version="1.1.0",
        request="GetFeature",
        typeName="MarineRegions:eez",
        outputFormat="json",
        filter=f"<Filter><PropertyIsEqualTo><PropertyName>mrgid_eez</PropertyName><Literal>{mgrid}</Literal></PropertyIsEqualTo></Filter>",
    )
    print(
        f"<Filter><PropertyIsEqualTo><PropertyName>mrgid_eez</PropertyName><Literal>{mgrid}</Literal></PropertyIsEqualTo></Filter>"
    )

    # Fetch data from WFS using requests
    response_eez = requests.get(url, params=params)

    # Check for successful request
    if response_eez.status_code == 200:
        data = response_eez.json()
    else:
        print(f"Error: {response_eez.status_code}")
    if data["totalFeatures"] != 1:
        raise ValueError(f"Expected 1 feature, got {data['totalFeatures']}\n: {data}")
    return gpd.GeoDataFrame.from_features(data["features"])


eez = fetch_maritime_eez("chinese")
eez.set_crs(epsg=4326, inplace=True)
eez.plot()

In [ ]:
prov_centroids = province_shapes.copy()
prov_centroids.geometry = prov_centroids.centroid

In [ ]:
voronoi_cells = gpd.GeoDataFrame(
    geometry=province_shapes.centroid.voronoi_polygons(), crs=province_shapes.crs
)
province_shapes = province_shapes.rename(columns={"region": "province"})
prov_centroids = province_shapes.copy()
prov_centroids.geometry = prov_centroids.centroid
voronoi_cells = voronoi_cells.sjoin(
    prov_centroids, predicate="contains"
)  # .set_index("province")
# {'intersects', None, 'contains_properly', 'within', 'dwithin', 'touches', 'contains', 'overlaps', 'crosses', 'covered_by', 'covers'}
# check the below with ez.overlay(voronoi_cells, how="intersection").boundary.plot()
voronoi_cells.shape

In [ ]:
if "index_right" in voronoi_cells.columns:
    voronoi_cells.drop(columns=["index_right"], inplace=True)

In [ ]:
voronoi_cells.overlay(eez, how="intersection")[["province", "geometry"]].set_index(
    "province"
).plot()

In [ ]:
voronoi_cells.shape

In [ ]:
gpd.read_file(
    "/home/ivanra/documents/PyPSA-China-PIK/resources/derived_data/regions/provinves_offshore.geojson"
)

In [ ]:
gpd.GeoDataFrame.from_features(data["features"]).plot()

In [ ]:
gpd_eez = gpd.GeoDataFrame([{"geometry": wkt_loads(wkt_s)}])
fig, ax = plt.subplots()
gpd_eez.set_crs(CRS, inplace=True)
gpd_eez.plot(ax=ax)

In [ ]:
gpd_eez.convex_hull.difference(china_shape).plot()

In [ ]:
geom.split("MULTIPOLYGON")[-1]

In [ ]:
from shapely.wkt import loads as wkt_loads

# sanitize the WKT string
wkt_s = "POLYGON" + geom.split("POLYGON")[-1].split('"')[0]
fig, ax = plt.subplots()
gpd.read_file(
    "/home/ivanra/documents/PyPSA-China-PIK/resources/data/regions/regions_offshore.geojson"
).plot(ax=ax, color="black")
eez = gpd.GeoDataFrame([{"geometry": wkt_loads(wkt_s)}])
eez.plot(ax=ax, color="red")
fetch_natural_earth_shape("admin_0_countries", "ADMIN", "China").plot(ax=ax)
fetch_natural_earth_shape("admin_0_countries", "ADMIN", "Taiwan").plot(
    ax=ax, color="grey"
)
fetch_natural_earth_shape("admin_0_countries", "ADMIN", "North Korea").plot(
    ax=ax, color="grey"
)
fetch_natural_earth_shape("admin_0_countries", "ADMIN", "South Korea").plot(
    ax=ax, color="grey"
)
fetch_natural_earth_shape("admin_0_countries", "ADMIN", "Japan").plot(
    ax=ax, color="grey"
)
gpd_eez.convex_hull.difference(china_shape).plot(ax=ax, color="green", alpha=0.5)